In [13]:
import pandas as pd 
string_df = pd.read_csv('map_string.csv')
string_map = dict(zip(string_df['PROTEIN'], string_df['SYMBOL']))
string_map = {k: v for k, v in string_map.items() if pd.notna(v)}

In [14]:
print("Number of items in string_map:", len(string_map))
print("First 10 items in string_map:", list(string_map.items())[:10])

Number of items in string_map: 20457
First 10 items in string_map: [('9606.ENSP00000376213', 'KIAA1683'), ('9606.ENSP00000464271', 'ENSG00000265690'), ('9606.ENSP00000472357', 'FLJ00297'), ('9606.ENSP00000007390', 'TSR3'), ('9606.ENSP00000000233', 'ARF5'), ('9606.ENSP00000259467', 'PDCL'), ('9606.ENSP00000006275', 'TRAPPC6A'), ('9606.ENSP00000006526', 'WDR54'), ('9606.ENSP00000006658', 'SPATA20'), ('9606.ENSP00000006724', 'CEACAM7')]


In [15]:
inet_df = pd.read_csv('map_inet.csv')
inet_map = dict(zip(inet_df['ENTREZID'].astype(str), inet_df['SYMBOL']))
inet_map = {k: v for k, v in inet_map.items() if pd.notna(v)}

In [16]:
print("Number of items in inet_map:", len(inet_map))
print("First 10 items in inet_map:", list(inet_map.items())[:10])

Number of items in inet_map: 19294
First 10 items in inet_map: [('1', 'A1BG'), ('29974', 'A1CF'), ('2', 'A2M'), ('144568', 'A2ML1'), ('127550', 'A3GALT2'), ('53947', 'A4GALT'), ('51146', 'A4GNT'), ('8086', 'AAAS'), ('65985', 'AACS'), ('13', 'AADAC')]


In [17]:
maps = {"string": string_map, "inet": inet_map}

In [18]:
print(maps)

{'string': {'9606.ENSP00000376213': 'KIAA1683', '9606.ENSP00000464271': 'ENSG00000265690', '9606.ENSP00000472357': 'FLJ00297', '9606.ENSP00000007390': 'TSR3', '9606.ENSP00000000233': 'ARF5', '9606.ENSP00000259467': 'PDCL', '9606.ENSP00000006275': 'TRAPPC6A', '9606.ENSP00000006526': 'WDR54', '9606.ENSP00000006658': 'SPATA20', '9606.ENSP00000006724': 'CEACAM7', '9606.ENSP00000000412': 'M6PR', '9606.ENSP00000000442': 'ESRRA', '9606.ENSP00000001008': 'FKBP4', '9606.ENSP00000001146': 'CYP26B1', '9606.ENSP00000002125': 'NDUFAF7', '9606.ENSP00000002165': 'FUCA2', '9606.ENSP00000002596': 'HS3ST1', '9606.ENSP00000002829': 'SEMA3F', '9606.ENSP00000003084': 'CFTR', '9606.ENSP00000003100': 'CYP51A1', '9606.ENSP00000006777': 'RHBDD2', '9606.ENSP00000007264': 'RPUSD1', '9606.ENSP00000259469': 'RPL35', '9606.ENSP00000003302': 'USP28', '9606.ENSP00000004103': 'TMEM176A', '9606.ENSP00000004531': 'SLC7A2', '9606.ENSP00000004921': 'CCL18', '9606.ENSP00000004982': 'HSPB6', '9606.ENSP00000005178': 'PDK4', 

In [19]:
import pandas as pd

def remove_reverse_duplicates(plinks_dup):
    plinks = plinks_dup[plinks_dup.iloc[:, 0] < plinks_dup.iloc[:, 1]]
    return plinks

def read_plinks():
    fname = "plinks_string.csv"
    print("READING", fname, "...")
    plinks_dup = pd.read_csv(fname)
    plinks = remove_reverse_duplicates(plinks_dup)
    return plinks

plinks = read_plinks()

READING plinks_string.csv ...


In [20]:
print(plinks)

                     protein1              protein2  combined_score
0        9606.ENSP00000000233  9606.ENSP00000263431             260
1        9606.ENSP00000000233  9606.ENSP00000353863             164
2        9606.ENSP00000000233  9606.ENSP00000342026             159
3        9606.ENSP00000000233  9606.ENSP00000240874             194
4        9606.ENSP00000000233  9606.ENSP00000379847             164
...                       ...                   ...             ...
5676523  9606.ENSP00000473005  9606.ENSP00000473172             503
5676524  9606.ENSP00000473036  9606.ENSP00000473233             510
5676525  9606.ENSP00000473043  9606.ENSP00000473233             157
5676526  9606.ENSP00000473166  9606.ENSP00000473233             157
5676527  9606.ENSP00000473233  9606.ENSP00000473243             157

[5676528 rows x 3 columns]


In [21]:
import pandas as pd

def create_links(df_links, mapping):
    portion = 100000
    i = 0
    n = len(df_links)
    df_new_links = df_links.copy()
    df_new_links['symbol1'] = None
    df_new_links['symbol2'] = None

    while i < n:
        j = i + portion - 1
        if j >= n:
            j = n - 1

        print(f"range {i} to {j} ...")

        df_new_links.loc[i:j, 'symbol1'] = df_new_links.loc[i:j, 'column_name_1'].astype(str).map(mapping)
        df_new_links.loc[i:j, 'symbol2'] = df_new_links.loc[i:j, 'column_name_2'].astype(str).map(mapping)

        i += portion

    df_new_links = df_new_links[['symbol1', 'symbol2', 'combined_score']]
    n_old = len(df_new_links)
    df_new_links = df_new_links.dropna(subset=['symbol1', 'symbol2'])
    n_new = len(df_new_links)

    print(f"********* {n_old - n_new} NULL rows are removed!")

    return df_new_links



In [22]:
import pandas as pd
import os

def get_slinks(maps):
    slinks = {}
    for m in maps.keys():
        fname = f"slinks_{m}.csv"
        if os.path.exists(fname):
            print(f"READING FROM -> {fname}")
            slinks[m] = pd.read_csv(fname)
        else:
            print(f"CREATING -> {fname}")
            if m == "string":
                xlinks = read_plinks()  # Make sure this function is defined in your code
            else:
                xlinks = pd.read_csv("elinks_inet.csv")
            slinks[m] = create_links(xlinks, maps[m])  # Assuming 'create_links' function is defined in your code
            print(f"WRITING {fname}")
            slinks[m].to_csv(fname, index=False)
    return slinks

slinks=get_slinks(maps)




READING FROM -> slinks_string.csv
READING FROM -> slinks_inet.csv


In [23]:
print(slinks)

{'string':                  symbol1 symbol2  combined_score
0                   ARF5   PRKCG             260
1                   ARF5  ZNF148             164
2                   ARF5   PRDX6             159
3                   ARF5   KALRN             194
4                   ARF5  ZNF143             164
...                  ...     ...             ...
5676523             PIH1  RUVBL2             503
5676524           PNMAL2   SPI-B             510
5676525  ENSG00000269693   SPI-B             157
5676526  ENSG00000269237   SPI-B             157
5676527            SPI-B  ZNF788             157

[5676528 rows x 3 columns], 'inet':         symbol1 symbol2  combined_score
0          A1BG    ABAT        0.065223
1          A1BG    ABL1        0.036557
2          A1BG    ACHE        0.029448
3          A1BG     ACR        0.029273
4          A1BG   ACTA1        0.042220
...         ...     ...             ...
1048570    DAD1   EEF1D        0.071123
1048571    DAD1   EEF1G        0.102599
1048